## 1. ライブラリをインポート (Import library)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


GPUを確認 (Confirm GPU)

In [ ]:
!nvidia-smi

Sat Aug 10 01:30:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

Gemma2-2Bモデルをimport (Import the Gemma2-2B model)

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

In [ ]:
import time
import os
import random
import functools
import csv
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve

from datasets import Dataset, DatasetDict

## 2. livedoor-news-dataをインポート(import livedoor-news-data)


In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar -zxvf ldcc-20140209.tar.gz

In [ ]:
from pathlib import Path
datapath = Path('text/')
topics = [x.parts[-1] for x in datapath.iterdir() if x.is_dir()]
topics

['movie-enter',
 'it-life-hack',
 'kaden-channel',
 'sports-watch',
 'topic-news',
 'smax',
 'peachy',
 'livedoor-homme',
 'dokujo-tsushin']

予測のためのジャンルを選択 (Choose the genre to predict)

In [ ]:
topics=['movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news']

In [ ]:
from sklearn.datasets import load_files
dataset = load_files(datapath, categories=topics, encoding='utf8')
dataset.data = [doc.split('\n', 2)[2] for doc in dataset.data]
dataset.data[0]

'いよいよ明日！JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】\nいよいよ明日に迫った、2011年12月10日。この日は皆既月食が起こるということで多くの天体ファンが楽しみにしている。JAXA 宇宙教育センターは「みんなで皆既月食を観察しようキャンペーン」を開催する。このキャンペーンは、全国から皆既月食の観測写真やスケッチを募集するもの。投稿された写真やスケッチは、JAXA 宇宙教育センターの Web サイトで紹介される。\n\n明日は、太平洋側のエリアでは晴れる場所が多く、絶好の観察日和だということだ。なお、 月食の時間は以下の通りだ。\n\n部分食の始まり    21時45分\n皆既食の始まり    23時05分\n皆既食の最大       23時31分\n皆既食の終わり    23時58分\n部分食の終わり    01時18分\n\nせっかくのチャンス、空を見上げてみてはいかがだろう。\n\nみんなで皆既月食を観察しようキャンペーン（JAXA）\n    \n\n■関連記事\n・洗濯したければゲームをクリアしろ！\u3000イギリスの学生が発明した洗濯機が楽しすぎる【話題】\n・女性が好きなのはFacebookで男性が好きなのはTwitterのなぜ【話題】\n・デジタル一眼レフカメラは現在コスパ重視の傾向、来年は新製品ラッシュの予感！【話題】\n・コダックが大ピンチ！\u3000なんと毎月54億円の大赤字【話題】\n・高すぎる？\u3000独島警備隊のインターネットは毎月１５０万円以上！【話題】\n'

In [ ]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
len(dataset.data)

4279

In [ ]:
len(dataset.filenames)

4279

In [ ]:
dataset.filenames[0].split('/')[1]

'kaden-channel'

In [ ]:
X = pd.DataFrame(dataset.data, columns=['input'])
X[0:1]

,input
0,いよいよ明日！JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】\nいよいよ...


In [ ]:
y = pd.DataFrame(dataset.filenames, columns=['output2'])
y[0:3]

,output2
0,text/kaden-channel/kaden-channel-6104126.txt
1,text/it-life-hack/it-life-hack-6380568.txt
2,text/sports-watch/sports-watch-5156792.txt


In [ ]:
y['output']=y['output2'].apply(lambda x: x.split('/')[1])
y=y[['output']]
y[0:3]

,output
0,kaden-channel
1,it-life-hack
2,sports-watch


In [ ]:
X.size, y.size

(4279, 4279)

## 3. 訓練データと検証データを作成　(Prepare training and valid data)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.7, test_size=0.3, random_state=100)
len(X_train), len(y_train), len(X_valid), len(y_valid)

(2995, 2995, 1284, 1284)

In [ ]:
pd.set_option("display.max_colwidth", 5000)

In [ ]:
y_train.nunique()

,0
output,5


In [ ]:
y_valid.nunique()

,0
output,5


## 4. Alpacaスタイルのデータセット作成 (Cretate Alpaca style data set)

In [ ]:
data_train = pd.concat([y_train, X_train], axis=1)
data_train["instruction"]= "You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'"
data_train = data_train.reindex(columns=['instruction','input','output'])
data_train[0:3]

,instruction,input,output
2741,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'",まだまだある“キングカズ”の知られざるエピソード\nTBS「タカトシの時間ですよ」（27日深夜放送）では、「ぶっちゃけ症候群SP」として、ゲストにサッカー解説者でタレントの武田修宏や元光GENJI・諸星和己が出演。“キングカズ”三浦知良と親交のある諸星は、キングの知られざるエピソードを紹介した。\n\n「カズは人間として大好きな人。僕の地元の隣の隣町なんですよ。で、タケチンは僕の隣町なんです。すごい仲いいんです。僕はカズから可愛がって貰って。で、僕が丁度（光GENJIを）解散して、事務所を辞める辞めないという時に（カズは）セリエAに行ったんです」と話し始めた諸星は、「電話かかってきたんです。“お前、悩んでるんだろ。ちょっと来い”っていって全部チケットも手配されて、当時ジェノバ所属の時。僕を一人イタリアに呼んでくれて。で、色々メッセージをくれるんですよ」と続けた。\n\nすると、諸星は「車でモナコとか連れてってくれる。色んなところに連れてってくれてモナコの王子様に会わせてくれる」「ミラノに連れていかれて（カズが愛用するホテル）フォーシーズンズってホテル行ったら、テーマが流れるんですよ。ゴットファーザーの。“カズが来た”って。そのくらい有名なんです」と、カズのキングたる所以といってもいい、イタリアでの思い出を明かす。\n\nまた、悩みを抱える諸星に対して「お前はこれからどうしていくのか。やって後悔することと、やらなくて後悔することがある。お前はどっち取る」と投げかけたというカズ。諸星は、そんなカズから送られた一連の激励に「やって後悔した方がいいと思って、色んなことにチャレンジした。それは全部カズ、キングカズのお陰」と語った。\n,sports-watch
3108,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'",天才少年トーマス・ホーン「日本のマンガは読まない」\n 9・11によって突然父を失った少年オスカーの「最愛の父が遺した鍵に託された最後のメッセージを探す旅」を描いた、希望と感動のヒューマンドラマ『ものすごくうるさくて、ありえないほど近い』。現地時間15日の18時、ニューヨーク・プレミアイベントが行われ、サンドラ・ブロックをはじめ、監督・キャスト陣が登場し、会場を沸かせた。\n\n 本作の主人公オスカー役を演じたのは、天才少年と呼び声の高いトーマス・ホーン。日本の物で好きなものを聞いてみると「日本のマンガは読まないけれど、日本食はとっても好きなんだ。」と答えた。ぜひ来日の際には、日本のマンガ文化にも触れて欲しいものである。\n\n スティーブン・ダルドリー監督は、トーマスについて「本当にファンタスティックだった。とっても才能があって、撮影期間中も素晴らしいの一言だった。彼のおかげで随分撮影も楽しかったし、彼とサンドラのシーンは感情表現が難しかったけれど、いいものになったと充実感を感じているよ。」と自信を見せる。\n\n アカデミー賞有力候補の呼び声も高かったが、12月米国公開と作品の完成が遅かったことが災いとなったか、第69回ゴールデン・グローブ賞に全くノミネートされなかった。トム・ハンクスやサンドラ・ブロックのアカデミー賞受賞者をキャストに迎えたこの作品がこれからどの様に挽回していくのか見ものである。\n\n 日本では、2012年2月18日（土）丸の内ピカデリー他、全国ロードショーとなる。\n\n\n\n【 MOVIE ENTER おすすめ記事 】\n・『ヒミズ』原作古谷実の描き下ろしイラストを解禁\n・どん底の男を支えた“本当の強さをもつ”ヒロインに、多くの女性が共感\n・制服姿の大政絢やナルニアの王子などにも直撃＜2011年インタビューまとめ vol.1＞\n・【いちおう妖ヶ劇場】第3話：「悪魔（グレムリン）＝日本人」は、あくまで噂ですからの巻\n,movie-enter
3337,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'",もっと便利な語学学習を！！NHK出版の「語学プレーヤー」【iPhoneでチャンスを掴め】\nNHK教育では、語学系の番組がたくさんありますよね。その教科書テキストはNHK出版で購入することができますが、音声のCDが付いているとはいえ、CDプレーヤーを持ち歩くというのも、いまではほとんどなくなってしまいました。そこで各番組の音声を聞くことができるiPhoneアプリを紹介します。こんな人におすすめ：\n語学学習の際ディクテーションや聞き取りをしたい人、NHKの語学教材を使っている人\nこんなときにおすすめ：\nゆっくり再生したり、部分繰り返ししたいとき\n\nアプリ名：語学プレーヤー〈NHK出版〉\nカテゴリ：教育\n価格：無料\n\n語学学習をしていて聞き取りたいのに早すぎて聞き取れないとか、ディクテーションするときだけ遅くしたいとか再生スピードの問題ありませんか？\nあるいは、ワンフレーズだけ繰り返し聞き取りをしたいとか。\nそんな方におすすめの「語学プレーヤー〈NHK出版〉」アプリです。\nNHK出版の教材を購入できます。無料サンプルでお試ししてみましょう！\n\n耳マークが着いている所で、速度を調節できます。\n他にも、指定した秒数（2、4、8、16、30秒）の巻き戻し/早送りができたり、\n区間指定の繰り返しもできます。\n\n\nもちろん自身でお持ちの語学教材も利用できます。\n\nプレイリストに追加して、あとはNHK語学教材と使い方は同じです。\n早さ調節や、区間指定繰り返しできます。\nこのアプリで、語学学習が便利になりそうですね。\n\n■RainbowApps - iPhoneアプリ開発を学ぶ\nなら1000人が受講したEagleが運営する東京校が一番！\n■成長率 888% 毎日 50万台増える Androidアプリに参入しよう。Androidマスターコース\n■人気のAngryBirds風アプリやスーパーマリオ風アプリが簡単に作れちゃう。ゲームアプリコース\n■iPhoneの日米人気アプリランキングをチェックするからRainbowApps\n\n■RainbowAppsの記事をもっと読む\n,it-life-hack


In [ ]:
data_valid = pd.concat([y_valid, X_valid], axis=1)
data_valid["instruction"]= "You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'"
data_valid=data_valid.reindex(columns=['instruction','input','output'])
data_valid[0:3]

,instruction,input,output
2883,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'",日ハム・大嶋匠、連日の活躍にも「少年野球以来なので、野球をするのは」\n10日放送、TBS「NEWS23クロス」では、同局・枡田絵理奈アナウンサーが、北海道日本ハムファイターズのキャンプ地を訪れ、早稲田大学ソフトボール部出身という異色の経歴を持ったプロ野球選手＝大嶋匠にインタビューを行った模様を伝えた。\n\n8日の紅白戦では、初打席で初ホームランを放ち、その翌日にも安打を打っている大嶋。ホームランについては「狙ってはなかったですけど、真っ直ぐだけ張って、思い切り振ろうしか考えてなかった」といい、打った瞬間は「センターフライかと思って、僕結構必死に一塁に走ってるんですけど」と苦笑い。\n\n枡田アナが「ソフトボールから硬式の野球になって、バッティングしていて違うところは？」と尋ねるも、「あんまり……。僕自身は感じない」とアッサリと答えた大嶋。ボールの見え方についても、「言葉にするのは難しいですね。なんていうんですかね。あるんすよ、やっぱ違いあるんすよ。ソフトボールってズドンとくるんですけど、野球はビューンってくるんですよ。伝わりますか？」と笑顔で語る。\n\nさらに「少年野球以来なので、野球をするのは。毎日新しい発見ばかりで楽しくキャンプしています」という大嶋は、「バッティングもしっかり継続してアピールしていきたいですし、守備の方でも、大嶋守れるんだって。チームに必要とされる存在になりたい」と意気込んだ。\n,sports-watch
962,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'","実験みたい！？あっという間に水道水から炭酸水を作るおもしろソーダメーカー【売れ筋チェック】\n疲れたとき炭酸水が飲みたくなるのはナゼだろう。風邪の引きはじめのちょっと調子が悪いときなども、\nあのシュワシュワッとした喉ごしで元気が出る気がする。実際、炭酸水には体内の血行を良くし、疲労回復する効果があるという。でも、突然無性に炭酸が飲みたくなっても、外が寒くなってきた最近はコンビニに買いに行くのもおっくうだ。「水道水が炭酸水になったらいいのに」という夢を少し現実にしてくれるのが、SodaStreamのソーダメーカーだ。\n■理科の実験みたい！電気を使わず水道水があっという間に炭酸水に。\nソーダメーカーで炭酸水を作るのはとても簡単。まず、水道水を専用ボトルに注ぐ。次にソーダメーカーに専用ボトルをセット。ガスを注入すれば出来上がり。まるで理科の実験みたいだ。グラスに注げばそのまますぐに飲めるが、 好みのシロップを入れれば炭酸飲料も作れる。シロップの濃さや炭酸の強さを自由に調整できるので、自分好みのオリジナル炭酸飲料が楽しめる。\n\n■便利なスターターキット、シロップは6種類\nスターターキットは、5種類のデザインから選べるソーダメーカー本体とガスシリンダー60リットル用1本、ソーダメーカー専用ボトル(1リットル)1本、ソーダメーカーとガスシリンダーの各取扱説明書がセットになっているので、届いたらすぐに炭酸水が作れる。また、シロップは定番のコーラ、ジンジャーエールの他レモンティ、レモン・ライム、パッションマンゴ、キウイ・ペアの6種類。どんな味になるのか全種類試してみたくなる。\n\n■ソーダメーカー スターターキット販売価格(税込み)\n*Jet(ブラック、ホワイト) 12,600円 商品コード：SSM1001〜SSM1002\n*Genesis(ブラック、レッド) 14,700円 商品コード：SSM1003〜SSM1004\n*Pure 18,900円 商品コード：SSM1005\n*Fizz(レッド、ブラック、ホワイト) 22.050円 商品コード：SSM1006〜SSM1008\n*Penguin 35,700円 商品コード：SSM1009 SSM1009\n\nSoda Stream\n\n \n（牧田 亜紀子）\n \n\n■関連記事\n・オフィスでツボ押しリラックス！ 小型の温感マッサージャーはパナソニック「プチリフレ」【売れ筋チェック】\n・第8回上映 中野 森 監督作品 『今日は秋刀魚の日』【ビデオSALON】\n・「着る電気毛布」—広電(KODEN) ホットローブCWS-X180LA【売れ筋チェック】\n・一晩平均３０通！？ アメリカのティーンはケータイしすぎで睡眠不足【話題】\n・【11月号記事連動】ソニーNEX-VG20でCEATECを取材する【ビデオSALON】\n",kaden-channel
2911,"You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'",正論。でもしんどいときもある！孫社長「人生は己の判断で決めるべき」\nソフトバンクグループの代表 孫正義氏は、Twitterを通じて活発な発言をしている。そんな同氏の格言的な言葉がネットで話題となっている。\n\nそれは、突然のつぶやきから始まった。\n\n孫社長は、\n「己の人生は己の判断で決めるべきだ。 けっして他人の判断に任せてはならない。」と、つぶやいた。\n\nこれに対して、下記のコメントがついている。\n「「判断」する際は仏の教えに従い決めましょう！常に『三毒』克服出来ているか問いかけ行動しよう！ 」\n「騙すような売り方してる人が偉そうなことばかり言うのはやめて。」\n「ぜひ秋山監督にサンドバッグを送ってあげてください！」\n「ジョブズさんが言った「自分の内面に耳を澄ませよ」と通じ合う話ですね。」\n「当然です。」\n「そらそうだ」\n「正論。でもしんどいときもある」\n「というかこんな事より災害情報でしょ」\n「だったら自然エネルギーの買い取り価格は国民に決めさせろよ。お前がごり押しで決めたくせに」\n「個人的にはiPhoneやiPadといったアップル製品って他人から使い方を指定されるツールの代表格だと思ってますが。。」\n「そういうことだ。iPadをもっていないからといって、卑屈になる必要は全くない。堂々と自分の人生を生きればよい。」\n「己の髪の毛の量が多いか少ないかは他人の判断で決めるべきだ。 けっして自分の判断に任せてはならない。（笑）」\n「孫さんあなたらしくていいよ!!実は隠れファン♪いけ〜ホークス♪今、関係ないか…」\n「ならiPhoneやiPadを持ってなくても人生悔い改める必要は全くありませんね。」\n\n様々な意見が寄せられている。\n\n■孫正義 (masason) on Twitter\n\n\n\n\n■孫正義氏に関連した記事を読む\n・笹食ってる場合じゃねぇ！孫社長 イベントの講演を宣伝\n・語れば語るほど、メッキが剥がれていく！孫社長「あきらめていないか？」\n・本当に稼動するのですね。凄い！孫社長 メガソーラーの状況を報告\n・おめでとうございます！孫社長 王会長との記念写真を掲載\n・じゃあ、その分の電気使わないでね！孫社長「大飯原発、運転継続理解できない」\n\nあんぽん 孫正義伝\n著者：佐野 眞一\n小学館(2012-01-10)\n販売元：Amazon.co.jp\nクチコミを見る\n,it-life-hack


In [ ]:
n=0
m=100
df_train = data_train.iloc[:m,:]
df_val = data_valid.iloc[:1000,:]
#df_test = data_valid.iloc[m:m+1000,:]
print(df_train.shape, df_val.shape)

(100, 3) (1000, 3)


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_train)
dataset

Dataset({
    features: ['instruction', 'input', 'output', '__index_level_0__'],
    num_rows: 100
})

## 5. モデルをダウンロード (Download the model)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## 6. モデルを訓練 (Training the model)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20,
        #max_steps = None,
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#@title メモリーの状況を表示　(Show current memory stats)
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.57 GB of memory reserved.


In [ ]:
%%time
trainer_stats = trainer.train()

Step,Training Loss
1,3.261900
2,3.085700
3,3.154700
4,2.994700
5,3.022500
6,3.052900
7,2.833700
8,2.678000
9,2.673300
10,2.498400


CPU times: user 16min 4s, sys: 10min 31s, total: 26min 36s
Wall time: 27min 3s


In [ ]:
#@title メモリーの最終の状況を表示　(Show final memory and time stats)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1621.3139 seconds used for training.
27.02 minutes used for training.
Peak reserved memory = 7.301 GB.
Peak reserved memory for training = 4.731 GB.
Peak reserved memory % of max memory = 49.505 %.
Peak reserved memory for training % of max memory = 32.079 %.


## 7. 精度検証　(Evaluate the model)

In [ ]:
article="いよいよ明日！JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】\nいよいよ明日に迫った、2011年12月10日。この日は皆既月食が起こるということで多くの天体ファンが楽しみにしている。JAXA 宇宙教育センターは「みんなで皆既月食を観察しようキャンペーン」を開催する。このキャンペーンは、全国から皆既月食の観測写真やスケッチを募集するもの。投稿された写真やスケッチは、JAXA 宇宙教育センターの Web サイトで紹介される。\n\n明日は、太平洋側のエリアでは晴れる場所が多く、絶好の観察日和だということだ。なお、 月食の時間は以下の通りだ。\n\n部分食の始まり    21時45分\n皆既食の始まり    23時05分\n皆既食の最大       23時31分\n皆既食の終わり    23時58分\n部分食の終わり    01時18分\n\nせっかくのチャンス、空を見上げてみてはいかがだろう。\n\nみんなで皆既月食を観察しようキャンペーン（JAXA）\n    \n\n■関連記事\n・洗濯したければゲームをクリアしろ！\u3000イギリスの学生が発明した洗濯機が楽しすぎる【話題】\n・女性が好きなのはFacebookで男性が好きなのはTwitterのなぜ【話題】\n・デジタル一眼レフカメラは現在コスパ重視の傾向、来年は新製品ラッシュの予感！【話題】\n・コダックが大ピンチ！\u3000なんと毎月54億円の大赤字【話題】\n・高すぎる？\u3000独島警備隊のインターネットは毎月１５０万円以上！"

In [ ]:
article

'いよいよ明日！JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】\nいよいよ明日に迫った、2011年12月10日。この日は皆既月食が起こるということで多くの天体ファンが楽しみにしている。JAXA 宇宙教育センターは「みんなで皆既月食を観察しようキャンペーン」を開催する。このキャンペーンは、全国から皆既月食の観測写真やスケッチを募集するもの。投稿された写真やスケッチは、JAXA 宇宙教育センターの Web サイトで紹介される。\n\n明日は、太平洋側のエリアでは晴れる場所が多く、絶好の観察日和だということだ。なお、 月食の時間は以下の通りだ。\n\n部分食の始まり    21時45分\n皆既食の始まり    23時05分\n皆既食の最大       23時31分\n皆既食の終わり    23時58分\n部分食の終わり    01時18分\n\nせっかくのチャンス、空を見上げてみてはいかがだろう。\n\nみんなで皆既月食を観察しようキャンペーン（JAXA）\n    \n\n■関連記事\n・洗濯したければゲームをクリアしろ！\u3000イギリスの学生が発明した洗濯機が楽しすぎる【話題】\n・女性が好きなのはFacebookで男性が好きなのはTwitterのなぜ【話題】\n・デジタル一眼レフカメラは現在コスパ重視の傾向、来年は新製品ラッシュの予感！【話題】\n・コダックが大ピンチ！\u3000なんと毎月54億円の大赤字【話題】\n・高すぎる？\u3000独島警備隊のインターネットは毎月１５０万円以上！'

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'", # instruction
        article, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out=tokenizer.batch_decode(outputs)
ans1=(out[0].split('###'))[-1]
ans2=ans1.split('\n')[1]
ans3=ans2.split('<')[0]
ans3

'kaden-channel'

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'", # instruction
        article, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out=tokenizer.batch_decode(outputs)
out

["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'\n\n### Input:\nいよいよ明日！JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】\nいよいよ明日に迫った、2011年12月10日。この日は皆既月食が起こるということで多くの天体ファンが楽しみにしている。JAXA 宇宙教育センターは「みんなで皆既月食を観察しようキャンペーン」を開催する。このキャンペーンは、全国から皆既月食の観測写真やスケッチを募集するもの。投稿された写真やスケッチは、JAXA 宇宙教育センターの Web サイトで紹介される。\n\n明日は、太平洋側のエリアでは晴れる場所が多く、絶好の観察日和だということだ。なお、 月食の時間は以下の通りだ。\n\n部分食の始まり    21時45分\n皆既食の始まり    23時05分\n皆既食の最大       23時31分\n皆既食の終わり    23時58分\n部分食の終わり    01時18分\n\nせっかくのチャンス、空を見上げてみてはいかがだろう。\n\nみんなで皆既月食を観察しようキャンペーン（JAXA）\n    \n\n■関連記事\n・洗濯したければゲームをクリアしろ！\u3000イギリスの学生が発明した洗濯機が楽しすぎる【話題】\n・女性が好きなのはFacebookで男性が好きなのはTwitterのなぜ【話題】\n・デジタル一眼レフカメラは現在コスパ重視の傾向、来年は新製品ラッシュの予感！【話題】\n・コダックが大

In [ ]:
ans1=(out[0].split('###'))[-1]
ans2=ans1.split('\n')[1]
ans3=ans2.split('<')[0]
ans3

'kaden-channel'

In [ ]:
df_val["input"][0:1]

,input
2883,日ハム・大嶋匠、連日の活躍にも「少年野球以来なので、野球をするのは」\n10日放送、TBS「NEWS23クロス」では、同局・枡田絵理奈アナウンサーが、北海道日本ハムファイターズのキャンプ地を訪れ、早稲田大学ソフトボール部出身という異色の経歴を持ったプロ野球選手＝大嶋匠にインタビューを行った模様を伝えた。\n\n8日の紅白戦では、初打席で初ホームランを放ち、その翌日にも安打を打っている大嶋。ホームランについては「狙ってはなかったですけど、真っ直ぐだけ張って、思い切り振ろうしか考えてなかった」といい、打った瞬間は「センターフライかと思って、僕結構必死に一塁に走ってるんですけど」と苦笑い。\n\n枡田アナが「ソフトボールから硬式の野球になって、バッティングしていて違うところは？」と尋ねるも、「あんまり……。僕自身は感じない」とアッサリと答えた大嶋。ボールの見え方についても、「言葉にするのは難しいですね。なんていうんですかね。あるんすよ、やっぱ違いあるんすよ。ソフトボールってズドンとくるんですけど、野球はビューンってくるんですよ。伝わりますか？」と笑顔で語る。\n\nさらに「少年野球以来なので、野球をするのは。毎日新しい発見ばかりで楽しくキャンプしています」という大嶋は、「バッティングもしっかり継続してアピールしていきたいですし、守備の方でも、大嶋守れるんだって。チームに必要とされる存在になりたい」と意気込んだ。\n


In [ ]:
L1 = list(df_val["input"])

t=time.time()
answer=[]
for i in range(len(L1)):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "You are a master at understanding what product is mentioned in the articles. You can choose only one product from the list : 'movie-enter','kaden-channel','sports-watch','it-life-hack','topic-news'", # instruction
            L1[i], # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    out=tokenizer.batch_decode(outputs)
    ans1=(out[0].split('###'))[-1]
    ans2=ans1.split('\n')[1]
    ans3=ans2.split('<')[0]
    answer.append(ans3)

t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict')

508.26413 Seconds to predict


In [ ]:
label=list(df_val['output'])

print("result:", answer)
print("label:", label)

result: ['sports-watch', 'kaden-channel', 'life-hack', 'movie-enter', 'kaden-channel', 'sports-watch', 'kaden-channel', 'kaden-channel', 'topic-news', 'sports-watch', 'topic-news', 'movie-enter', 'movie-enter', 'it-life-hack', 'kaden-channel', 'movie-enter', 'it-life-hack', 'movie-enter', 'kaden-channel', 'it-life-hack', 'topic-news', 'topic-news', 'sports-watch', 'sports-watch', 'it-life-hack', 'sports-watch', 'sports-watch', 'sports-watch', 'sports-watch', 'kaden-channel', 'it-life-hack', 'it-life-hack', 'it-life-hack', 'it-life-hack', 'it-life-hack', 'sports-watch', 'hulu-enter', 'sports-watch', 'topic-news', 'topic-news', 'movie-enter', 'it-life-hack', 'movie-enter', 'topic-news', 'movie-enter', 'movie-enter', 'sports-watch', 'movie-enter', 'topic-news', 'sports-watch', 'movie-enter', 'movie-enter', 'movie-enter', 'sports-watch', 'kaden-channel', 'topic-news', 'kaden-channel', 'movie-enter', 'it-life-hack', 'movie-enter', 'topic-news', 'kaden-channel', 'yomuca', 'sports-watch', 'mo

In [ ]:
sdf=pd.Series(answer).value_counts()
sdf.sort_index()

,count
produto,1
ATEM1M/E,1
HVL-AVS2.0,1
app-enter,1
book-enter,1
dv-enter,1
enter,1
enter-news,4
excel,2
hulu-enter,1


In [ ]:
labeldf=pd.Series(label).value_counts()
labeldf.sort_index()

,count
it-life-hack,198
kaden-channel,211
movie-enter,194
sports-watch,216
topic-news,181


In [ ]:
print('accuracy score: {0:0.4f}'.format(accuracy_score(label, answer)))

accuracy score: 0.8150


In [ ]:
print(classification_report(label, answer, digits=4))

                            precision    recall  f1-score   support

                   produto     0.0000    0.0000    0.0000         0
                  ATEM1M/E     0.0000    0.0000    0.0000         0
                HVL-AVS2.0     0.0000    0.0000    0.0000         0
                 app-enter     0.0000    0.0000    0.0000         0
                book-enter     0.0000    0.0000    0.0000         0
                  dv-enter     0.0000    0.0000    0.0000         0
                     enter     0.0000    0.0000    0.0000         0
                enter-news     0.0000    0.0000    0.0000         0
                     excel     0.0000    0.0000    0.0000         0
                hulu-enter     0.0000    0.0000    0.0000         0
              it-life-hack     0.7119    0.8737    0.7846       198
             kaden-channel     0.8344    0.5972    0.6961       211
                 life-hack     0.0000    0.0000    0.0000         0
               movie-enter     0.9282    0.8660

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
